<a href="https://colab.research.google.com/github/srikarraju/eGrocery_Demand_Prediction/blob/main/Reinforce_1_eGrocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 133kB 6.5MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
from torch.distributions import Categorical
import math

In [ ]:
class EGroceryEnv():

    def __init__(self, df=pd.DataFrame({0: [0]}), products_count=10, features=['a'], shelf_life=[1], wastage_cost=[1],
                 shortage_cost=[1]):
        super(EGroceryEnv, self).__init__()

        self.df = df
        self.products_count = products_count
        self.shelf_life = shelf_life
        self.features = features
        self.wastage_cost = wastage_cost
        self.shortage_cost = shortage_cost
        self.current_step = 0
        self.wastage_track = list([])
        self.shortage_track = list([])
        self.reward_track = list([])

        # variables to track shartage and wastage
        self.shortage = np.array(list([0] * self.products_count))
        self.wastage = np.array(list([0] * self.products_count))

        # Define Stock
        self.stock = list([])
        for i in range(self.products_count):
            self.stock.append([0])
        print(features)

    def _next_observation(self):

        obs = self.df.loc[self.current_step, self.features]

        st_temp = list([])
        for i in range(len(self.stock)):
            for j in range(1, min(int(self.stock[i][0]) + 1, 5)):
                st_temp.append(self.stock[i][j])
            if (self.stock[i][0] == 5):
                st_temp.append(self.stock[i][4])
            elif (self.stock[i][0] < 5):
                for j in range(int(self.stock[i][0]) + 1, 6):
                    st_temp.append(0)
            else:
                st_temp.append(np.sum(self.stock[i][5:int(self.stock[i][0]) + 1]))
        obs = list(obs) + list(st_temp) + list(self.shelf_life)
        return obs

    def _take_action(self, action):
        # Add products to the current stocks
        for i in range(self.products_count):
            if (len(self.stock[i]) < self.shelf_life[i]):
                for j in range(len(self.stock[i]), int(self.shelf_life[i])):
                    self.stock[i].append(0)
            self.stock[i].append(action[i])
            self.stock[i][0] = self.shelf_life[i]

        # Fullfill demand
        prods = ['prod' + str(i) for i in [8, 11, 15, 17, 94, 95, 96, 110, 112, 128]]
        demand = self.df.loc[self.current_step + 1, prods]
        for i in range(self.products_count):
            for j in range(1, int(self.stock[i][0]) + 1):
                if (self.stock[i][j] >= demand[i]):
                    self.stock[i][j] = self.stock[i][j] - demand[i]
                    demand[i] = 0
                    break
                else:
                    demand[i] = demand[i] - self.stock[i][j]
                    self.stock[i][j] = 0
            if (demand[i] > 0):
                self.shortage[i] = demand[i]

        # Update shelf life and find out wastage
        for i in range(self.products_count):
            self.stock[i][0] = self.stock[i][0] - 1
            if (self.stock[i][1] > 0):
                self.wastage[i] = self.stock[i][1]
            for j in range(1, int(self.stock[i][0]) + 1):
                self.stock[i][j] = self.stock[i][j + 1]
            self.stock[i].pop()

    def step(self, action):
        # update stock, fullfill demand and calculate shortage and wastage
        quantity = [6, 10, 15, 4, 6, 2, 7, 50, 2, 30]
        action1 = [0] * self.products_count
        for i in range(len(action)):
            action1[i] = int(action[i]) * quantity[i]
        self._take_action(action1)
        self.action = action

        # increment step
        self.current_step += 1

        reward = -1 * (np.matmul(self.wastage_cost, self.wastage.transpose()) + np.matmul(self.shortage_cost,
                                                                                          self.shortage.transpose()))
        self.reward = reward
        done = (self.current_step < 0) or (self.current_step > self.df.shape[0] - 2)

        obs = self._next_observation()

        self.wastage_track.append(np.sum(self.wastage))
        self.shortage_track.append(np.sum(self.shortage))
        self.reward_track.append(np.abs(self.reward))

        self.shortage = np.array(list([0] * self.products_count))
        self.wastage = np.array(list([0] * self.products_count))

        return obs, np.sum(self.reward), done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.current_step = 0
        self.shortage = np.array(list([0] * self.products_count))
        self.wastage = np.array(list([0] * self.products_count))
        self.stock = list([])
        for i in range(self.products_count):
            self.stock.append([0])

        # print(len(self.features),products_count)
        # print(self.features)
        return [0] * len(self.features) + [0] * 6 * self.products_count

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std, lr, betas):
        super(ActorCritic, self).__init__()
        # actor
        self.actor = nn.Sequential(
            nn.Linear(state_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Softmax())
        self.action_var = torch.full((action_dim,), action_std * action_std)
        self.actor_optimizer = torch.optim.SGD(self.actor.parameters(), lr=lr)
        self.MseLoss = nn.MSELoss()
        self.policy_net_loss = torch.rand(700)

    def forward(self, state):
        probs = self.actor(state)

        return probs

    def act(self, state):
        state = torch.Tensor(state)
        probs = self.actor(state)
        dist = Categorical(probs)
        action = dist.sample()
        action_logprob = dist.log_prob(action)

        return action.detach()

    def evaluate(self, state, action):
        state = torch.Tensor(state)
        probs = self.actor(state)
        dist = Categorical(probs)
        action = dist.sample()
        action_logprob = dist.log_prob(action)
        dist_entropy = dist.entropy()

        return action_logprob, dist_entropy

    def select_action(self, state, memory):
        state = torch.FloatTensor(state.reshape(1, -1))
        return self.policy_old.act(state, memory).cpu().data.numpy().flatten()

    def update(self, states, actions, rewards,next_states,lr,gamma):
        rewards = np.array(rewards)
        R = torch.tensor([np.sum(rewards[i:] * (gamma ** np.array(range(0, len(rewards) - i)))) for i in
                          range(len(rewards))]).float()
        states = torch.tensor(states).float()
        actions = torch.tensor(actions)

        log_probs, _ = self.evaluate(states, actions)

        self.policy_net_loss = -lr*torch.sum(log_probs *R)
        self.actor_optimizer.zero_grad()
        self.policy_net_loss.backward()
        self.actor_optimizer.step()

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_trainx.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_testx.csv')

#products_count = 10
products_count = 1

# avg_f7 = ['prod'+str(i)+'avg7' for i in [8,11,15,17,94,95,96,110,112,128]]
# avg_f15 = ['prod'+str(i)+'avg15' for i in [8,11,15,17,94,95,96,110,112,128]]
# avg_f30 = ['prod'+str(i)+'avg30' for i in [8,11,15,17,94,95,96,110,112,128]]

avg_f7 = ['prod'+str(i)+'avg7' for i in [11]]
avg_f15 = ['prod'+str(i)+'avg15' for i in [11]]
avg_f30 = ['prod'+str(i)+'avg30' for i in [11]]

features = ['month', 'monthday', 'weekday'] + avg_f7 + avg_f15 + avg_f30

print(features)

#shelf_life = np.array([4, 3, 5, 10, 7, 2, 1, 3, 8, 6], dtype=np.float32)
shelf_life = np.array([3], dtype=np.float32)

wastage_cost = np.array([1]*products_count, dtype=np.float16)
shortage_cost = np.array([2]*products_count, dtype=np.float16)

action_std = 0.1
eps_clip = 0.2
gamma = 0.99

lr = 1
betas = (0.9, 0.999)
K_epochs = 5

update_timestep = 20
time_step=0
running_reward = 0

state_dim = len(features) + 6*products_count
action_dim = products_count

env = EGroceryEnv(df_train, products_count, features, shelf_life, wastage_cost, shortage_cost)
writer = SummaryWriter()

Total_reward = []

['month', 'monthday', 'weekday', 'prod11avg7', 'prod11avg15', 'prod11avg30']
['month', 'monthday', 'weekday', 'prod11avg7', 'prod11avg15', 'prod11avg30']


In [ ]:
policy = ActorCritic(state_dim, action_dim, action_std, lr, betas)
gamma = 0.75

n_episode = 1
lr = 0.00001
returns = []
while n_episode <=50000:
    rewards,states,actions,next_states = [],[],[],[]
    state = env.reset()
    trajectory = []
    while True:
      action = policy.act(torch.tensor(state).float())
      new_state, reward, done, info = env.step([action.item()])
      action = action.tolist()

      states.append(state)
      next_states.append(new_state)
      actions.append(action)
      rewards.append(reward)

      state = new_state
      if done==True:
          break

    policy.update(states,actions,rewards,next_states,lr,gamma)
    returns.append(np.sum(rewards))
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.sum(rewards)))
    n_episode += 1

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Streaming output truncated to the last 5000 lines.
Episode:   2441	Avg. Return: -35269.00
Episode:   2442	Avg. Return: -35275.00
Episode:   2443	Avg. Return: -35298.00
Episode:   2444	Avg. Return: -35292.00
Episode:   2445	Avg. Return: -35286.00
Episode:   2446	Avg. Return: -35263.00
Episode:   2447	Avg. Return: -35298.00
Episode:   2448	Avg. Return: -35298.00
Episode:   2449	Avg. Return: -35298.00
Episode:   2450	Avg. Return: -35263.00
Episode:   2451	Avg. Return: -35286.00
Episode:   2452	Avg. Return: -35274.00
Episode:   2453	Avg. Return: -35263.00
Episode:   2454	Avg. Return: -35287.00
Episode:   2455	Avg. Return: -35263.00
Episode:   2456	Avg. Return: -35298.00
Episode:   2457	Avg. Return: -35298.00
Episode:   2458	Avg. Return: -35269.00
Episode:   2459	Avg. Return: -35263.00
Episode:   2460	Avg. Return: -35298.00
Episode:   2461	Avg. Return: -35280.00
Episode:   2462	Avg. Return: -35286.00
Episode:   2463	Avg. Return: -35298.00
Episode:   2464	Avg. Return: -35269.00
Episode:   24

In [ ]:
print(actions)

[0, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
env2 = EGroceryEnv(df_test, products_count, features, shelf_life, wastage_cost, shortage_cost)
state = env2.reset()
actions_test = []
rewards_test = []
for step in range(0,30):
  #print("state:",state)
  action = policy.act(torch.tensor(state).float())
  new_state, reward, done, info = env2.step([action.item()])
  actions_test.append(action.item())
  rewards_test.append(reward)
  #print(reward)
  state = new_state

print(rewards_test)

['month', 'monthday', 'weekday', 'prod11avg7', 'prod11avg15', 'prod11avg30']
[0.0, 0.0, 0.0, 0.0, -4.0, -94.0, -88.0, -16.0, -90.0, -6.0, 0.0, -12.0, 0.0, 0.0, 0.0, -80.0, -66.0, -62.0, -42.0, -48.0, -46.0, -8.0, -14.0, -14.0, -10.0, -26.0, -2.0, -28.0, -28.0, -42.0]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
print(df_test['prod11'].tolist())
print(len(df_test['prod11'].tolist()))

[0.0, 45.12000000000001, 33.62, 40.57, 32.805, 35.955, 31.84, 23.25, 37.02, 53.525, 19.3, 17.05, 32.75, 34.19, 18.75, 19.005, 62.0, 51.605, 30.0, 46.27, 39.75, 39.0, 25.75, 24.25, 11.75, 3.0, 25.5, 30.0, 22.5, 40.13, 41.3]
31


In [ ]:
print(env2.wastage_track)
print(env2.shortage_track)
print(sum(rewards_test))
print(actions_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 47, 44, 8, 45, 3, 0, 6, 0, 0, 0, 40, 33, 31, 21, 24, 23, 4, 7, 7, 5, 13, 1, 14, 14, 21]
-826.0
[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
